# Project Description: "Needs to be completed"

## How to load and format data was taking from: https://colab.research.google.com/github/trekhleb/machine-learning-experiments/blob/master/experiments/rock_paper_scissors_cnn/rock_paper_scissors_cnn.ipynb#scrollTo=DJ8jGFnTLt8t

### Un-comment and run if your environment is missing tensorflow datasets package.

In [ ]:
# ! pip install tensorflow_datasets

In [29]:
import tensorflow as tf
import tensorflow_datasets as tfds

In [30]:
# Load the Rock Paper Scissors dataset
dataset = tfds.load('rock_paper_scissors')

# Split the dataset into training and testing sets
train_dataset = dataset['train']
test_dataset = dataset['test']

In [31]:
# number of samples to reduce size of model
#num_samples = 1000

# Take a subset of the training dataset
#train_dataset = train_dataset.take(num_samples)

### Data Exploration: "Needs to be completed"

### Pre-Process data

In [32]:
# Image size reduced from 300x300 to 100x100
def preprocess_image(data):
    image = tf.image.resize(data['image'], (128, 128))
    image = tf.cast(image, tf.float32) / 255.0
    label = data['label']
    return image, label

train_dataset = train_dataset.map(preprocess_image)
train_dataset = train_dataset.batch(32).prefetch(tf.data.AUTOTUNE)

test_dataset = test_dataset.map(preprocess_image)
test_dataset = test_dataset.batch(32).prefetch(tf.data.AUTOTUNE)

In [33]:
# Model Layers reduced to 5, so it can work with streamlit
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])

In [34]:
# we can also add learning rate to optimizer
# adam_optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)


# Compile cnn model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [35]:

# Train cnn mmodel
model.fit(train_dataset, epochs=10)

Epoch 1/10
79/79 [==============================] - 8s 81ms/step - loss: 1.4404 - accuracy: 0.3429
Epoch 2/10
79/79 [==============================] - 1s 15ms/step - loss: 1.0987 - accuracy: 0.3333
Epoch 3/10
79/79 [==============================] - 1s 15ms/step - loss: 1.0987 - accuracy: 0.3333
Epoch 4/10
79/79 [==============================] - 1s 11ms/step - loss: 1.0987 - accuracy: 0.3333
Epoch 5/10
79/79 [==============================] - 1s 11ms/step - loss: 1.0987 - accuracy: 0.3333
Epoch 6/10
79/79 [==============================] - 1s 14ms/step - loss: 1.0987 - accuracy: 0.3210
Epoch 7/10
79/79 [==============================] - 1s 18ms/step - loss: 1.0987 - accuracy: 0.3214
Epoch 8/10
79/79 [==============================] - 1s 12ms/step - loss: 1.0987 - accuracy: 0.3040
Epoch 9/10
79/79 [==============================] - 1s 10ms/step - loss: 1.0987 - accuracy: 0.3087
Epoch 10/10
79/79 [==============================] - 1s 10ms/step - loss: 1.0987 - accuracy: 0.3155


In [36]:
# Evaluate cnn model
model.evaluate(test_dataset)

12/12 [==============================] - 1s 63ms/step - loss: 1.0986 - accuracy: 0.3333


[1.0986144542694092, 0.3333333432674408]

In [37]:
# save cnn model
model_name = 'rock_paper_scissors_cnn.h5'
model.save(model_name, save_format='h5')

## Finetunning: (Needs to be done)
### Train data is overfitting and test accuracy is low. Change parameters. See attached google notebook for optimal parameters. Also, pre-processing may need to be optimized as well, pre-processing imput shape, etc. Again, see notebook from google colab referenced at beginning of notebook.